# Sample Notebook for exploring gnomAD in BigQuery
This notebook contains sample queries to explore the gnomAD dataset which is hosted through the Google Cloud Public Datasets Program.

## Setup and Authentication

If you just want to look at sample results, you can scroll down to see the output of the existing queries without having to run anything. If you would like to re-run the queries or make changes, you will need to authenticate as your user and set the Google Cloud project in which to run the analysis.

In [1]:
# Import libraries
import os

# Imports for using and authenticating BigQuery
from google.colab import auth
%load_ext google.cloud.bigquery

### User Authentication
Before running any queries using BigQuery, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Google identity account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter. This should be a Google account which you can login to and which has access to run BigQuery jobs in the Google Cloud project specified in the next step.

In [2]:
auth.authenticate_user()

### Set Google Cloud Project
To run queries in BigQuery, you need to specify the Google Cloud project that will be used. The first 1 TB of query data processed per month is free. For more details, see the [BigQuery Pricing](https://cloud.google.com/bigquery/pricing) page.

To find your Project ID, go to the [Project Settings page](https://console.cloud.google.com/iam-admin/settings) in the Google [Cloud Console](https://console.cloud.google.com/). You can select the project you want using the drop-down menu at the top of the page.

In [3]:
# Replace project_id with your Google Cloud Project ID. 
os.environ["GOOGLE_CLOUD_PROJECT"]='project_id'

# gnomAD Queries
## Query1
Find 10,000 SNV on chr17 that are more common in women than men, min sample size set to 30.

In [4]:
%%bigquery result1
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_female AS AC_f,
       alternate_bases.AC_male AS AC_m,
       ROUND(alternate_bases.AC_female / alternate_bases.AC, 3) AS f_ratio
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE alternate_bases.AC > 30 AND vep.SYMBOL IS NOT NULL
ORDER BY f_ratio DESC
LIMIT 10000


In [5]:
result1.head()

,str_pos,ref,alt,type,gene,f_type,AC,AC_f,AC_m,f_ratio
0,79140505,A,C,snv,AATK,Transcript,41,39,2,0.951
1,79140505,A,C,snv,AATK-AS1,Transcript,41,39,2,0.951
2,33998749,T,TC,ins,AP2B1,Transcript,37,35,2,0.946
3,7221462,T,G,snv,GPS2,Transcript,80,75,5,0.938
4,7221462,T,G,snv,NEURL4,Transcript,80,75,5,0.938


We can condensed the result and only list gene symbols and the number of variants found in the query1.

In [6]:
result1.groupby('gene').count()[['str_pos']].sort_values(by=['str_pos'], ascending=False).head()

,str_pos
gene,
DNAH17,72
CTC-297N7.11,65
RP11-799N11.1,61
RNF213,58
DNAH9,54


## Query2
Find top 1,000 SNV on chr17 that show the most significant differences between male samples of African-American ancestry versus Finnish ancestry

In [7]:
%%bigquery result2
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC_male AS AC_m,
       alternate_bases.AC_fin_male AS AC_fin_m,
       alternate_bases.AC_afr_male AS AC_afr_m,
       ROUND(ABS(alternate_bases.AC_fin_male - alternate_bases.AC_afr_male) / alternate_bases.AC_male, 3) AS fin_afr_diff
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND
      alternate_bases.AC_male > 20 AND alternate_bases.AC_fin_male > 0 AND alternate_bases.AC_afr_male > 0
order by fin_afr_diff DESC
LIMIT 1000

In [8]:
result2.head()

,str_pos,ref,alt,type,gene,f_type,AC_m,AC_fin_m,AC_afr_m,fin_afr_diff
0,76557038,G,A,snv,DNAH17,Transcript,56,53,1,0.929
1,76116856,C,T,snv,TMC6,Transcript,247,228,1,0.919
2,6900268,G,A,snv,RP11-589P10.5,Transcript,34,32,1,0.912
3,6900268,G,A,snv,AC027763.2,Transcript,34,32,1,0.912
4,6900268,G,A,snv,ALOX12,Transcript,34,32,1,0.912


## Query3
Find top 1000 genes with the highest number of SNV on chr17

In [9]:
%%bigquery result3
SELECT gene, count(1) AS num_snv
FROM
(
SELECT DISTINCT 
       start_position AS str_pos,
       alternate_bases.alt AS alt,
       vep.SYMBOL AS gene,
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND alternate_bases.allele_type = 'snv'
)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1000

In [10]:
result3.head()

,gene,num_snv
0,CTC-297N7.11,9589
1,DNAH17,9208
2,RP11-799N11.1,9190
3,RNF213,6561
4,DNAH2,6361
